<a href="https://colab.research.google.com/github/bryaanabraham/Wandering_Guardian/blob/main/ML%20Implementation/Sequential_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [47]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM

In [48]:
data = pd.read_csv("training_dataset.csv")
data = data.dropna(how='all')
data.head()

,longitude,latitude,path_id
0,12.821062,80.044729,1.0
1,12.821219,80.044743,1.0
2,12.821479,80.044736,1.0
3,12.821561,80.044743,1.0
4,12.821636,80.044743,1.0


In [49]:
path = []
for i in range(1,len(data['path_id'].unique())+1):
    path.append(data[data['path_id'] == i])

In [50]:
X = data[['longitude', 'latitude']].values
y = data['path_id'].values

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

model = Sequential([
    Dense(256, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(256, activation='relu'),
    Dense(128, activation='relu'),
    Dense(64, activation='relu'),
    Dense(len(np.unique(y))+1, activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.fit(X_train, y_train, epochs=20, batch_size=32, validation_split=0.2, verbose=0)

loss, accuracy = model.evaluate(X_test, y_test)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)

for i in range(len(path)):
    new_X = path[i][['longitude', 'latitude']].values
    new_X_scaled = scaler.transform(new_X)
    predicted_probabilities = model.predict(new_X_scaled)
    predicted_path_ids = np.argmax(predicted_probabilities, axis=1)

    print(f"Predicted Paths IDs for path '{i+1}'", end=": ")
    for path_id in predicted_path_ids:
        print(path_id, end=", ")
    print()
    mode_path_id = np.argmax(np.bincount(predicted_path_ids))
    print(f"Mode of Predicted Paths: {mode_path_id}", end="\n\n")


2/2 [==============================] - 0s 9ms/step - loss: 1.0022 - accuracy: 0.6579
Test Loss: 1.0021556615829468
Test Accuracy: 0.6578947305679321
2/2 [==============================] - 0s 7ms/step
Predicted Paths IDs for path '1': 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 5, 3, 3, 3, 3, 3, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 3, 
Mode of Predicted Paths: 2

1/1 [==============================] - 0s 23ms/step
Predicted Paths IDs for path '2': 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 5, 5, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 2, 3, 
Mode of Predicted Paths: 2

1/1 [==============================] - 0s 27ms/step
Predicted Paths IDs for path '3': 2, 2, 2, 2, 2, 2, 2, 2, 2, 5, 3, 3, 3, 3, 3, 3, 3, 3, 2, 2, 1, 2, 2, 3, 5, 5, 5, 
Mode of Predicted Paths: 2

1/1 [==============================] - 0s 23ms/step
Predicted Paths IDs for path '4': 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 3, 
Mode of Predicted Paths: 4

2/2 [=========

In [51]:
# import numpy as np
# import pandas as pd
# from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import StandardScaler
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Dense
# from sklearn.metrics import accuracy_score

# scaler = StandardScaler()
# X_scaled = scaler.fit_transform(X)

# X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# layer_configs = [
#     [128, 128, 64, 32],
#     [256, 128, 64, 32],
#     [64, 64, 32, 16],
#     [256, 256, 128, 64],
#     [512, 256, 128, 64]
# ]

# for i, layers in enumerate(layer_configs):
#     model = Sequential()
#     model.add(Dense(layers[0], activation='relu', input_shape=(X_train.shape[1],)))
#     for units in layers[1:]:
#         model.add(Dense(units, activation='relu'))
#     model.add(Dense(len(np.unique(y))+1, activation='softmax'))

#     model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

#     model.fit(X_train, y_train, epochs=20, batch_size=32, validation_split=0.2, verbose=0)

#     loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
#     print(f"Configuration {i+1}: Layers: {layers}, Test Accuracy: {accuracy}")


In [52]:
model.save_weights("model_weights.h5")
model_json = model.to_json()

with open("model.json", "w") as json_file:
    json_file.write(model_json)

In [53]:
from tensorflow.keras.models import model_from_json
from sklearn.preprocessing import MinMaxScaler

with open('model.json', 'r') as json_file:
    loaded_model_json = json_file.read()

model = model_from_json(loaded_model_json)
model.load_weights('model_weights.h5')

input_data = path[1][['longitude', 'latitude']].values

scaler = MinMaxScaler()
scaler.fit(data[['longitude', 'latitude']].values)

for i in range(len(path)):
    new_X = path[i][['longitude', 'latitude']].values
    new_X_scaled = scaler.transform(new_X)
    predicted_probabilities = model.predict(new_X_scaled)
    predicted_path_ids = np.argmax(predicted_probabilities, axis=1)

    print(f"Predicted Paths IDs for path '{i+1}'", end=": ")
    for path_id in predicted_path_ids:
        print(path_id, end=", ")
    print()
    mode_path_id = np.argmax(np.bincount(predicted_path_ids))
    print(f"Mode of Predicted Paths: {mode_path_id}", end="\n\n")


2/2 [==============================] - 0s 7ms/step
Predicted Paths IDs for path '1': 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 
Mode of Predicted Paths: 3

1/1 [==============================] - 0s 25ms/step
Predicted Paths IDs for path '2': 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 2, 2, 2, 2, 2, 3, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 
Mode of Predicted Paths: 3

1/1 [==============================] - 0s 28ms/step
Predicted Paths IDs for path '3': 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 2, 2, 2, 2, 2, 3, 1, 1, 1, 
Mode of Predicted Paths: 3

1/1 [==============================] - 0s 21ms/step
Predicted Paths IDs for path '4': 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 
Mode of Predicted Paths: 3

2/2 [==============================] - 0s 6ms/step
Predicted Paths IDs for path '5': 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 